In [3]:
from ccdc.io import CrystalReader

crystal_reader = CrystalReader(r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\T2_9_experimental_crystals\DEBXIT01_1478358_T2gamma_240K.cif', format='cif')
cryst0 = crystal_reader[0]
crystal_reader = CrystalReader(r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\T2_9_experimental_crystals\T2epsilon.cif', format='cif')
cryst1 = crystal_reader[0]

In [4]:
from ccdc.crystal import PackingSimilarity

PS = PackingSimilarity()
comp = PS.compare(cryst0,cryst1)
print(comp.rmsd)

0.1833274334455433


In [5]:
import os
'''
Initial run 9x9 with duplicate comparisons: 516.1s

'''
directory = r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\T2_9_experimental_crystals'
PS = PackingSimilarity()
i=0

for outter_cif in os.listdir(directory):
    print(i)
    print(outter_cif)

    crystal_reader = CrystalReader(directory+'\\'+outter_cif)
    outter_crystal = crystal_reader[0]

    for inner_cif in os.listdir(directory):
        if inner_cif != outter_cif:
            print(inner_cif)

            crystal_reader = CrystalReader(directory+'\\'+inner_cif)
            inner_crystal = crystal_reader[0]
            comp = PS.compare(outter_crystal,inner_crystal)
            print(comp.rmsd)
        else:
            print('match')
    i+=1

0
DEBXIT01_1478358_T2gamma_240K.cif
match
DEBXIT02_1478361_T2gamma_450K.cif
0.008311771166553638
DEBXIT03_1478363_T2gamma_300K.cif


KeyboardInterrupt: 

In [12]:
import os
import numpy as np
import pandas as pd
from ccdc.io import CrystalReader
from ccdc.crystal import PackingSimilarity
'''
Initial run 9x9 with duplicate comparisons: 516.1s

Run without duplicate comparison 270.4s
'''
directory = r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\T2_9_experimental_crystals'
PS = PackingSimilarity()
i=0
cifs = []


for filename in os.listdir(directory):
    cifs.append(filename)
#rmsd_matrix = np.empty((len(cifs),len(cifs)))
rmsd_matrix = np.empty((9,9))
rmsd_matrix[:] = np.nan
print('MATRIX: ')
print(rmsd_matrix)
print(len(cifs))
cols = cifs
for i in range(len(cifs)):
    outter_cif = cifs[i]
    print(i)
    print(outter_cif)

    crystal_reader = CrystalReader(directory+'\\'+outter_cif)
    outter_crystal = crystal_reader[0]
    for j in range(len(cifs)):
        inner_cif = cifs[j]
        if inner_cif != outter_cif and inner_cif != "":
            print(outter_cif +' / '+inner_cif)
            
            crystal_reader = CrystalReader(directory+'\\'+inner_cif)
            inner_crystal = crystal_reader[0]
            comp = PS.compare(outter_crystal,inner_crystal)
            rmsd = comp.rmsd
            #print('column: ',i)
            #print('row: ',j)
            #print('rmsd: ',rmsd)
            rmsd_matrix[i][j] = rmsd
            #print(rmsd_matrix)
        elif inner_cif == outter_cif:
            print(outter_cif +' / '+inner_cif + ' MATCH')
            rmsd_matrix[i][j] = 0
    cifs[i] = ""
    i+=1
df = pd.DataFrame(rmsd_matrix, columns=cols, index=cols)
print(df)

MATRIX: 
[[nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]]
9
0
DEBXIT01_1478358_T2gamma_240K.cif
DEBXIT01_1478358_T2gamma_240K.cif / DEBXIT01_1478358_T2gamma_240K.cif MATCH
DEBXIT01_1478358_T2gamma_240K.cif / DEBXIT02_1478361_T2gamma_450K.cif
column:  0
row:  1
rmsd:  0.008311771166553638
[[       nan 0.00831177        nan        nan        nan        nan
         nan        nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan     

In [13]:
from tabulate import tabulate
print(tabulate(df, headers=cols, tablefmt='psql'))

+----+-----+--------------+--------------+-------------+------------+-------------+-----------+------------+------------+
|    |     |              |              |             |            |             |           |            |            |
|----+-----+--------------+--------------+-------------+------------+-------------+-----------+------------+------------|
|    | nan |   0.00831177 |   0.00317795 |   0.0349603 |   0.451601 |   0.439513  |   2.79007 |   0.426636 |   0.183327 |
|    | nan | nan          |   0.00742361 |   0.0341313 |   0.452272 |   0.440206  |   2.79051 |   0.426416 |   0.183868 |
|    | nan | nan          | nan          |   0.0330806 |   0.453086 |   0.44099   |   2.79176 |   0.427092 |   0.183369 |
|    | nan | nan          | nan          | nan         |   0.462409 |   0.450179  |   2.80467 |   0.427149 |   0.183587 |
|    | nan | nan          | nan          | nan         | nan        |   0.0547681 |   0.24631 |   0.664421 |   0.434904 |
|    | nan | nan        